In [11]:
from permutation import perm
import rubik_rotations as rubik
from tqdm.notebook import tqdm

searchfor = rubik.rubik("search",
    perm.getIdentity(8),
    perm.fromCycle([rubik.cUF, rubik.cUB], length=12),
    perm.getIdentity(24),
    perm.fromCycle([rubik.FU, rubik.UB, rubik.UF, rubik.BU], [rubik.RU, rubik.UR], length=24),
    perm.getIdentity(6))

def quality(op):
    def wrappedQuality(ll):
        qua = 0
        for item in ll:
            if isinstance(item, str):
                if item[0].upper() in {"U", "F", "R"}:
                    qua += 5
                elif item[0].upper() in {"M", "S", "E"}:
                    qua += 2
            else:
                qua += wrappedQuality(item[0]) * item[1]
        return qua
    return wrappedQuality(op._array[0])

output = []
def sidePerm(op):
    opp = op.angle().order()
    maxorder = (op**opp).side().order() * opp
    maxpower = 30 // (len(op.opers())) + 1
    for p in range(opp, min(maxorder, maxpower), opp):
        pos = op ** p
        if pos.coreSide().distinct() <= 4:
            inlist = False
            for key in rubik._norm_dict:
                rotpos = rubik._norm_dict[key].getInv() * pos * rubik._norm_dict[key]
                reflpos = rotpos.getReflect("Z")
                reflpos._array[5] = rotpos._array[5]
                if rotpos in output:
                    inlist = True
                    index = output.index(rotpos)
                    break
                if reflpos in output:
                    inlist = True
                    index = output.index(reflpos)
                    break
            if inlist:
                lenex, lennew = len(output[index]), len(pos)
                if lenex > lennew:
                    output[index] = pos
                elif lenex == lennew:
                    powex = output[index]._array[0][0][1]
                    if powex < p:
                        output[index] = pos
                    elif powex == p and quality(output[index]) < quality(pos):
                        output[index] = pos
            else:
                output.append(pos)

doStuff = sidePerm

operations = rubik.basOpers
for op1 in tqdm(operations):
    for op2 in operations:
        if rubik.sameKind(op1, op2):
            continue
        ops2 = op1*op2
        doStuff(ops2)
        
        for op3 in operations:
            if rubik.sameKind(op2, op3):
                continue
            ops3 = ops2 * op3
            doStuff(ops3)

            for op4 in operations:
                if rubik.sameKind(op3, op4):
                    continue
                ops4 = ops3 * op4
                doStuff(ops4)
                
                for op5 in operations:
                    if rubik.sameKind(op4, op5):
                        continue
                    ops5 = ops4 * op5
                    doStuff(ops5)

def sortOperOut(op):
    strlist = op.opers()[0]
    return op.coreSide().distinct(),  len(strlist[0]) * strlist[1]
output.sort(key=sortOperOut)
for posit in output:
    stringcoreside = str(posit.coreSide())
    for i in range(11, -1, -1):
        stringcoreside = stringcoreside.replace(str(i), rubik.decode_core_side[i])

    stringside = str(posit.side())
    for i in range(23, -1, -1):
        stringside = stringside.replace(str(i), rubik.decode_side[i])
    print(posit, "\n\t", stringcoreside, "\n\t", stringside, "\n",sep="")

  0%|          | 0/18 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
num4 = 0
for op in output[:36]:
    if op._array[0][0][1] == 4:
        print(repr(op))
        print()
        num4 += 1
print(num4)

(U F R2 f r)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (UF,FU) (UB,BU) (UL,LU) (RB,BR)
    Center: e

(u f U2 F R)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (UB,BU) (FR,RF) (RD,DR) (RB,BR)
    Center: e

(U F r u R2)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (FR,RF) (FD,DF) (FL,LF) (RD,DR)
    Center: e

(U R f u R2)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (UR,RU) (FD,DF) (FL,LF) (RB,BR)
    Center: e

(U F2 u f R)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (FR,RF) (FD,DF) (RD,DR) (RB,BR)
    Center: e

(U F2 u r F)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (UR,RU) (FL,LF) (RD,DR) (RB,BR)
    Center: e

(U2 r f U R)^4
Core angle: e
 Core side: e
     Angle: e
      Side: (UR,RU) (UB,BU) (FD,DF) (FL,LF)
    Center: e

7
